# Retrieve cds.climate.copernicus.eu/ data with the CDS API
# One variable per file, **netCDF-3** format (which is *not recommended*)
## ERA5-Land hourly data from 2015-01-01 to 2019-12-31
### cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=form
#
# Then saves the data in a local netCDF 
#
# Also saves it on EOSC/CESNET (requires credentials not disclosed here)

**ERA5-Land** is a reanalysis dataset providing a consistent view of the evolution of land variables over several decades at an enhanced resolution compared to ERA5. ERA5-Land has been produced by replaying the land component of the ECMWF ERA5 climate reanalysis. Reanalysis combines model data with observations from across the world into a globally complete and consistent dataset using the laws of physics. Reanalysis produces data that goes several decades back in time, providing an accurate description of the climate of the past.

In [ ]:
import pkg_resources
if not any(list(map(lambda i: i.key == 'cdsapi', pkg_resources.working_set))):
    !pip install cdsapi

In [ ]:
import cdsapi

In [ ]:
c = cdsapi.Client()

In [ ]:
def cds_download(path, year, month, variable):
    c.retrieve(
        'reanalysis-era5-land',
        {
            'variable': variable,
            'year': year,
            'month': month,
            'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',
                    '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
            'time': ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00',
                     '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
            'area': [71.18, 15.59, 68.35, 31.06],
            'format': 'netcdf'
        },
        path + variable + '_' + year + '_' + month + '.nc')
    return

In [ ]:
path = '/home/jovyan/data/nc/'

In [ ]:
variables =  ['2m_temperature', 'total_precipitation']
for year in range(2019, 2015 - 1, -1):
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        for variable in variables:
            print('cds_download ("' + path + 'reanalysis-era5-land_' + str(year) + '_' + month + '.nc", ["' + variable + '"])')
            #cds_download (path + 'reanalysis-era5-land_', str(year), month, variable)

# Combine individual files into a single file and save it to local disk

In [ ]:
import os
import glob
import xarray as xr

In [ ]:
path = '/home/jovyan/data/netCDF/'
filename = 'reanalysis-era5-land_*.nc'
files =  glob.glob(os.path.join(path, filename))
files.sort()
files

In [ ]:
ERA5land = xr.open_mfdataset(files)

In [ ]:
ERA5land

### Save ERA5-land data locally as a single netCDF file

In [ ]:
netCDF_name = 'reanalysis-era5-land_hourly_2015-01-01_2019-12-31_Troms-Finnmark_T2m-SD-TP.nc'
path = '/home/jovyan/data/'
ERA5land.to_netcdf(path + netCDF_name, engine='netcdf4')

# Define s3 storage parameters

In [ ]:
import s3fs
s3_path = 'Data'
s3_prefix =  "s3://" + s3_path
print(s3_prefix)
access_key = !aws configure get aws_access_key_id
access_key = access_key[0]
secret_key = !aws configure get aws_secret_access_key
secret_key = secret_key[0]
client_kwargs={'endpoint_url': 'https://object-store.cloud.muni.cz'}

### Define s3 store for the **netCDF file**

In [ ]:
url_netCDF = os.path.join(s3_prefix, os.path.basename(netCDF_name))
print(url_netCDF)

### Write in remote S3 storage at https://object.store.cloud.muni.cz **as netCDF**

In [ ]:
store_netCDF = s3fs.S3FileSystem(client_kwargs=client_kwargs, key=access_key, secret=secret_key)
print(path + netCDF_name, url_netCDF)
store_netCDF.put(path + netCDF_name, url_netCDF)

## Check that the data was effectively stored at https://object-store.cloud.muni.cz and can be read by anyone (anonymously)

In [ ]:
store = s3fs.S3FileSystem(anon=False, client_kwargs=client_kwargs)
store.ls('Data', detail=True, refresh=True)

In [ ]:
s3path = 'Data/reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.nc'

In [ ]:
ERA5land = xr.open_dataset(store.open(s3path))
ERA5land